<center>
<img src="https://drive.google.com/uc?id=1f1gGVI-rxcHjA90WEGNvvtSXF1pAxQwg" alt="Fasilkom UI" width="300"/>

CSGE603130 • Kecerdasan Artifisial dan Sains Data Dasar

Semester Ganjil 2022/2023

Fakultas Ilmu Komputer, Universitas Indonesia

##**Tugas *2*: *Classification***

###**Tenggat Waktu: 19 November 2022, 23.55 WIB**
</center>

####**Ketentuan:**

1. Dataset yang digunakan pada tugas ini beserta deskripsinya telah disediakan di SCeLe.
2. Buatlah program Jupyter Notebook yang menjawab pertanyaan sesuai dengan perintah soal yang disediakan.
3. Dokumen Jupyter Notebook yang telah dilengkapi dengan jawaban dikumpulkan dengan format penamaan **Kelas_Tugas2_NPM_Nama.ipynb**. Contoh: A_Tugas2_2006123456_Budi.ipynb
4. Kumpulkan dokumen tersebut pada submisi yang telah disediakan di SCeLe sesuai dengan kelas masing-masing sebelum **19 November 2022, 23.55 WIB**. Keterlambatan pengumpulan akan dikenakan pinalti.
5. Praktikum ini dirancang sebagai **tugas mandiri**. Plagiarisme tidak diperkenankan dalam bentuk apapun. Adapun kolaborasi berupa diskusi (tanpa menyalin) dan literasi masih diperbolehkan dengan mencantumkan kolaborator dan sumber.


## **Pernyataan Integritas**

Wajib diisi. Tanpa pernyataan integritas submisi akan dikenakan pinalti.

In [1]:
# Isi dengan data diri Anda
NAMA = "Ezra Pasha Ramadhansyah"
KELAS = "F"
NPM = "2006597872"

# Isi dengan NPM teman yang berdiskusi dengan Anda
KOLABORATOR = []

# Isi dengan sumber referensi yang Anda gunakan dalam mengerjakan
REFERENSI = []

In [2]:
PERNYATAAN_INTEGRITAS = "Saya, %s dari kelas %s dengan NPM %s, menyatakan bahwa seluruh jawaban pada pekerjaan ini murni saya kerjakan sendiri.\n\
Saya tidak mencontek jawaban, memberikan jawaban, maupun menyalin dari sumber manapun.\n \
\n\
Jika saya melanggar pernyataan tersebut, saya siap menerima konsekuensi apapun yang diberikan.\n   \
(%s)" % (NAMA, KELAS, NPM, NAMA)

print(PERNYATAAN_INTEGRITAS)

Saya, Ezra Pasha Ramadhansyah dari kelas F dengan NPM 2006597872, menyatakan bahwa seluruh jawaban pada pekerjaan ini murni saya kerjakan sendiri.
Saya tidak mencontek jawaban, memberikan jawaban, maupun menyalin dari sumber manapun.
 
Jika saya melanggar pernyataan tersebut, saya siap menerima konsekuensi apapun yang diberikan.
   (Ezra Pasha Ramadhansyah)


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import classification_report, precision_score, recall_score, classification_report, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV, train_test_split, KFold, cross_val_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB, CategoricalNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder, Normalizer
from sklearn.tree import DecisionTreeClassifier

## **Dataset**

Dataset yang digunakan adalah dataset Mobile Prices dimana kita memklasifikasi handphone lewat harganya, terdapat 21 atribut dan target atributnya adalah price_range.

Penjelesan atribut:
* battery_power : Total energy a battery can store in one time measured in mAh
* blue : Has bluetooth or not
* clock_speed : speed at which microprocessor executes instructions
* dual_sim : Has dual sim support or not
* fc : Front Camera mega pixels
* four_g : Has 4G or not
* int_memory : Internal Memory in Gigabytes
* m_dep : Mobile Depth in cm
* mobile_wt : Weight of mobile phone
* n_cores : Number of cores of processor
* pc : Primary Camera mega pixels
* px_height : Pixel Resolution Height
* px_width : Pixel Resolution Width
* ram : Random Access Memory in Mega Bytes
* sc_h : Screen Height of mobile in cm
* sc_w : Screen Width of mobile in cm
* talk_time : Longest time that a single battery charge will last when you are
* three_g : Has 3G or not
* touch_screen : Has touch screen or not
* wifi : Has wifi or not
* price_range : This is the target variable with value of 0(low cost), 1(medium cost), 2(high cost) and 3(very high cost).

Sumber: https://www.kaggle.com/datasets/iabhishekofficial/mobile-price-classification (dengan modifikasi)

## [25] **Preprocessing**

**Pada bagian ini Anda akan diminta untuk melakukan preprocessing dan eksplorasi data sebelum data digunakan dalam model klasifikasi.**

1\. [3] Berikan ringkasan mengenai data tersebut terkait dengan deskripsi setiap atribut, jumlah missing values, jumlah duplikasi data, dan kemungkinan adanya outliers pada data!

In [4]:
df = pd.read_csv('D:\Backup\Kuliah\KASDD files\mobile_phone.csv')

display(df.shape)
display(df.head())
print("\nDescription")
display(df.describe())
display(df['pc'].describe())
print("\nNull Data\n")
display(df.isna().sum())

print("Duplicated Data :", df.duplicated().sum())
print('\n')

def count_outlier(dataFrame):
    print("Check for Outlier\n")
    
    for col in dataFrame:
        Q1 = dataFrame[col].quantile(0.25)
        Q3 = dataFrame[col].quantile(0.75)
        IQR = Q3 - Q1
        LB = Q1 - IQR * 1.5
        UB = Q3 + IQR * 1.5

        print(col, dataFrame.loc[(dataFrame[col] < LB) | (dataFrame[col] > UB)][col].count())
        
count_outlier(df)

(2021, 21)

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7.0,0.6,188,2,...,20,756,2549.0,9,7,NaN,0,0.0,1.0,1
1,1021,1,0.5,1,0,1,53.0,0.7,136,3,...,905,1988,2631.0,17,3,7.0,1,1.0,0.0,2
2,563,1,0.5,1,2,1,41.0,0.9,145,5,...,1263,1716,2603.0,11,2,9.0,1,1.0,0.0,2
3,615,1,2.5,0,0,0,10.0,0.8,131,6,...,1216,1786,2769.0,16,8,11.0,1,0.0,0.0,2
4,1821,1,1.2,0,13,1,44.0,NaN,141,2,...,1208,1212,1411.0,8,2,15.0,1,1.0,0.0,1



Description


,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
count,2021.000000,2021.000000,1920.000000,2021.000000,2021.000000,2021.000000,1915.000000,1928.000000,2021.000000,2021.000000,...,2021.000000,2021.000000,1927.000000,2021.000000,2021.000000,1937.000000,2021.000000,1925.000000,1922.000000,2021.000000
mean,1239.205344,0.492331,1.521302,0.505690,4.303810,0.523998,32.086162,0.501556,140.272143,4.532905,...,646.858981,1249.240970,2120.320187,12.323602,5.790203,11.022716,0.762494,0.504935,0.506243,1.503216
std,439.916748,0.500065,0.816020,0.500091,4.330382,0.499547,18.198040,0.289261,35.393281,2.291511,...,445.159739,433.141437,1079.774853,4.213155,4.369054,5.459042,0.425660,0.500106,0.500091,1.114537
min,501.000000,0.000000,0.500000,0.000000,0.000000,0.000000,2.000000,0.100000,80.000000,1.000000,...,0.000000,500.000000,256.000000,5.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000
25%,852.000000,0.000000,0.700000,0.000000,1.000000,0.000000,16.000000,0.200000,109.000000,3.000000,...,284.000000,874.000000,1210.500000,9.000000,2.000000,6.000000,1.000000,0.000000,0.000000,1.000000
50%,1230.000000,0.000000,1.500000,1.000000,3.000000,1.000000,32.000000,0.500000,141.000000,4.000000,...,567.000000,1245.000000,2130.000000,12.000000,5.000000,11.000000,1.000000,1.000000,1.000000,2.000000
75%,1617.000000,1.000000,2.200000,1.000000,7.000000,1.000000,48.000000,0.800000,170.000000,7.000000,...,947.000000,1633.000000,3033.500000,16.000000,9.000000,16.000000,1.000000,1.000000,1.000000,2.000000
max,1998.000000,1.000000,3.000000,1.000000,19.000000,1.000000,64.000000,1.000000,200.000000,8.000000,...,1960.000000,1998.000000,3998.000000,19.000000,18.000000,20.000000,1.000000,1.000000,1.000000,3.000000


count    2021.000000
mean        9.938149
std         6.054671
min         0.000000
25%         5.000000
50%        10.000000
75%        15.000000
max        20.000000
Name: pc, dtype: float64


Null Data



battery_power      0
blue               0
clock_speed      101
dual_sim           0
fc                 0
four_g             0
int_memory       106
m_dep             93
mobile_wt          0
n_cores            0
pc                 0
px_height          0
px_width           0
ram               94
sc_h               0
sc_w               0
talk_time         84
three_g            0
touch_screen      96
wifi              99
price_range        0
dtype: int64

Duplicated Data : 21


Check for Outlier

battery_power 0
blue 0
clock_speed 0
dual_sim 0
fc 18
four_g 0
int_memory 0
m_dep 0
mobile_wt 0
n_cores 0
pc 0
px_height 2
px_width 0
ram 0
sc_h 0
sc_w 0
talk_time 0
three_g 480
touch_screen 0
wifi 0
price_range 0


2\. [3] Berdasarkan eksplorasi anda pada nomor 1, lakukan data preparation hingga data tersebut menurut anda cukup “clean” dan dapat memberikan hasil klasifikasi yang maksimal.

In [5]:
df.dropna(inplace=True)

def remove_outlier(col, df):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    LB = Q1 - IQR * 1.5
    UB = Q3 + IQR * 1.5

    df_filtered = df[(df[col] >= LB) & (df[col] <= UB)]
    return df_filtered
    
#df = remove_outlier('fc', df)
df = remove_outlier('px_height', df)

display(df.isna().sum())

count_outlier(df)

display(df.shape)

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

Check for Outlier

battery_power 0
blue 0
clock_speed 0
dual_sim 0
fc 11
four_g 0
int_memory 0
m_dep 0
mobile_wt 0
n_cores 0
pc 0
px_height 0
px_width 0
ram 0
sc_h 0
sc_w 0
talk_time 0
three_g 328
touch_screen 0
wifi 0
price_range 0


(1347, 21)

**Keterangan**<br>
Null data yang ada di-drop karena berjumlah terlalu banyak untuk direplace dengan central tendency. Selain itu menggantinya dengan central tendency juga bisa mengganggu distribusi data. <br><br>
Outlier pada `px_height` dan `fc` bisa diremove atau direplace. Setelah beberapa iterasi, dalam preprocessing ini saya memilih untuk remove outlier `px_height` dan membiarkan `fc`. Alasan remove adalah karena px_height menjadi fitur optimal. Sementara alasan membiarkan outlier fc adalah karena fc tidak menjadi fitur optimal. Menghapus outlier yang ada bisa mempengaruhi fitur yang lain sehingga dibiarkan. `three_g` dibiarkan karena kemungkinan besar memang bagian dari data dan bukan error.

3\. [3] Menurut Anda, apakah perlu dilakukan normalisasi terhadap data, atau cukup menggunakan data asli? Jika ya, bentuk normalisasi apa yang tepat digunakan pada data? Jelaskan secara singkat alasan Anda!


In [6]:
X_raw = df.drop(["price_range"],axis=1)
normalized = MinMaxScaler()
X = normalized.fit_transform(X_raw)

y = df["price_range"]

> Jawaban: 

> Normalisasi perlu dilakukan karena range yang dimiliki data cukup beragam. Dengan normalisasi kita mengubah mean menjadi 0 dan variansi menjadi 1. Selain itu kita juga akan melakukan fitting dengan **KNN** yang memerlukan data yang sudah discale

4\. [10] Sebelum data diproses lebih lanjut lakukan *feature selection* dengan memilih beberapa fitur yang paling optimal untuk mengklasifikasikan label target!

In [7]:
display(df.corr())


selector = SelectKBest(f_classif, k=4)
X_selected = selector.fit_transform(X, y)

print("Jumlah feature (sebelum dilakukan feature selection ) :", X.shape[1])
print("Jumlah feature (setelah dilakukan feature selection ) :", X_selected.shape[1])
print("Jumlah K didapatkan dari pengecekan korelasi")

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
battery_power,1.000000,0.003665,0.016820,-0.053470,0.016356,0.036925,0.005736,0.025960,-0.008130,-0.040219,...,0.009159,-0.027072,0.003993,-0.013099,-0.005747,0.040093,0.029253,0.004178,-0.015102,0.207743
blue,0.003665,1.000000,0.021083,0.008161,0.001862,0.020122,0.024580,0.036368,-0.002954,0.020826,...,-0.001896,-0.035930,0.020859,0.000504,-0.006129,0.024291,-0.032399,-0.021454,-0.030418,0.017862
clock_speed,0.016820,0.021083,1.000000,0.017061,0.000931,-0.010528,0.010888,-0.009088,-0.048459,0.010834,...,-0.016835,0.025212,0.008111,-0.020487,0.009927,-0.020319,-0.023414,0.021892,-0.011599,-0.001834
dual_sim,-0.053470,0.008161,0.017061,1.000000,-0.051256,-0.027413,-0.005203,-0.018392,-0.009159,-0.019179,...,-0.028550,0.003039,0.020362,0.014937,-0.016743,-0.021654,-0.032399,-0.051177,0.028976,-0.005037
fc,0.016356,0.001862,0.000931,-0.051256,1.000000,-0.045578,-0.017480,-0.008340,0.051546,0.000983,...,-0.043890,0.002647,0.001651,0.015221,-0.002808,0.003970,-0.004804,-0.007289,0.026430,0.006101
four_g,0.036925,0.020122,-0.010528,-0.027413,-0.045578,1.000000,0.015416,-0.010165,0.018056,-0.000069,...,-0.015819,0.002351,0.049973,0.054624,0.054797,-0.031515,0.584888,0.033650,-0.030747,0.056742
int_memory,0.005736,0.024580,0.010888,-0.005203,-0.017480,0.015416,1.000000,0.009582,-0.034782,-0.007883,...,0.018425,0.002594,0.068339,0.033848,0.024673,0.002199,0.007381,-0.052711,0.004161,0.084742
m_dep,0.025960,0.036368,-0.009088,-0.018392,-0.008340,-0.010165,0.009582,1.000000,0.049962,-0.005891,...,0.001435,-0.014775,-0.028524,-0.027673,-0.021956,0.023961,-0.028239,0.002467,-0.024193,-0.021449
mobile_wt,-0.008130,-0.002954,-0.048459,-0.009159,0.051546,0.018056,-0.034782,0.049962,1.000000,-0.014722,...,0.004058,0.000280,0.003625,-0.009687,0.008340,0.003654,0.034969,-0.027915,0.008608,-0.025916
n_cores,-0.040219,0.020826,0.010834,-0.019179,0.000983,-0.000069,-0.007883,-0.005891,-0.014722,1.000000,...,0.019907,0.009605,-0.015182,-0.019794,0.024832,0.024459,0.018584,0.050861,-0.022497,-0.004805


Jumlah feature (sebelum dilakukan feature selection ) : 20
Jumlah feature (setelah dilakukan feature selection ) : 4
Jumlah K didapatkan dari pengecekan korelasi


5\. [6] Apa saja fitur-fitur yang terpilih hasil dari *feature selection*? Jelaskan strategi Anda dalam memilih fitur-fitur tersebut dan berikan analisis mengapa fitur tersebut cocok digunakan untuk mengklasifikasi label target!

> Jawaban: 

> Yang diambil adalah `battery_power`, `ram`, `px_height`, dan `px_weight`. Keempatnya memiliki korelasi dengan price_range di atas 0.1 sehingga terpilih menjadi fitur

## [75] **Classification**

**Pada bagian ini Anda akan diminta untuk melakukan klasifikasi harga suatu mobile phone berdasarkan fitur-fitur yang sudah Anda pilih sebelumnya. Anda akan mencoba berbagai macam model klasifikasi seperti Decision Tree, Random Forest, Naive Bayes, dan K-nearest Neighbors untuk mencari model yang memiliki performa terbaik.**

### [3] Preparation

6\. [3] Buatlah model K-Fold Cross Validation yang akan digunakan untuk membagi dataset train dan test menjadi beberapa bagian (*fold*). Strategi ini akan Anda gunakan pada semua *classifier* dibandingkan dengan strategi sebelumnya yang hanya membagi data ke dalam data training dan testing menggunakan method `train_test_split`.

In [8]:
kf = KFold(n_splits=5,shuffle=True,random_state=5)

### [19] Decision Tree

7\. [4] Implementasikan model Decision Tree tanpa konfigurasi parameter apapun, serta berikan hasil evaluasinya!

In [9]:
dt = DecisionTreeClassifier()

np.mean(cross_val_score(dt, X_selected, y, cv=kf))

0.8589591078066915

8\. [4] Implementasikan model Decision Tree dengan memberikan parameter nilai `min_samples_leaf=50`; serta berikan hasil evaluasinya!

In [10]:
dt_50 = DecisionTreeClassifier(min_samples_leaf=50)

np.mean(cross_val_score(dt_50, X_selected, y, cv=kf))

0.7438276194410023

9\. [6] Bandingkan model yang dibuat pada soal nomor 7 dan nomor 8, manakah yang lebih baik? Berikan analisis Anda terkait bagaimana hubungan perubahan parameter tersebut dengan kualitas hasil klasifikasi!

> Jawaban: 

> Model pada nomor 7 lebih akurat dibandingkan nomor 8. Parameter `min_samples_leaf` menentukan minimal sample yang ada pada suatu leaf_node. Dengan mengeset parameter tersebut menjadi 50, model kemungkinan bersifat underfitted.

10\. [5] Buatlah model yang memiliki performa yang lebih bagus dibanding kedua model sebelumnya! Anda dibebaskan untuk melakukan eksplorasi dalam mengkonfigurasi parameter yang dipakai.

In [11]:
param = {
         'max_depth' : [5,10,20,25,None],
         'min_samples_leaf' : [1,2,3,4,5,6,7,8,10],
         'random_state' : [1,2,3,4,5]
        }

dt_optimimized = DecisionTreeClassifier()

estimator = GridSearchCV(estimator= dt_optimimized, 
                   param_grid=param, cv= kf)

estimator.fit(X_selected, y)

estimator.best_params_

{'max_depth': 20, 'min_samples_leaf': 2, 'random_state': 3}

In [12]:
dt_opt = DecisionTreeClassifier(**estimator.best_params_)

np.mean(cross_val_score(dt_opt, X_selected, y, cv=kf))

0.8663665152140988

### [18] Random Forest

11\. [4] Implementasikan model Random Forest tanpa konfigurasi parameter apapun, serta berikan hasil evaluasinya!

In [13]:
rf = RandomForestClassifier()

np.mean(cross_val_score(rf, X_selected, y, cv=kf))

0.90647115517004

12\. [8] Implementasikan model Random Forest dengan parameter yang didapatkan dari hasil Hyperparameter Tuning, serta berikan hasil evaluasinya!

In [14]:
param = {
         'random_state' : [10,100,150,175,190,195,200,205,210]
        }

rf_est = RandomForestClassifier()

estimator_rf = GridSearchCV(estimator= rf_est, 
                   param_grid=param, cv= kf)

estimator_rf.fit(X_selected, y)

estimator_rf.best_params_

{'random_state': 190}

In [15]:
rf_param = RandomForestClassifier(**estimator_rf.best_params_)

np.mean(cross_val_score(rf_param, X_selected, y, cv=kf))

0.913151590251962

13\. [6] Bandingkan model yang dibuat pada soal nomor 11 dan nomor 12, manakah yang lebih baik? Berikan analisis Anda terkait bagaimana hubungan perubahan parameter tersebut dengan kualitas hasil klasifikasi!

> Jawaban: 

> Karena hyperparameter yang dituning adalah random_state, susah ditebak apabila yang didapatkan adalah terbaik dari seluruh range. Tetapi bisa dipastikan bahwa model pada nomor 12 akan selalu konsisten lebih bagus dibandingkan nomor 11 karena hyperparameter random state tersebut


### [16] Naive Bayes

14\. [4] Implementasikan model dari salah satu variasi Naive Bayes; serta berikan hasil evaluasinya!

In [16]:
nb = GaussianNB()

np.mean(cross_val_score(nb, X_selected, y, cv=kf))

0.7846867685529396

15\. [6] Apa alasan dan pertimbangan Anda dalam memilih variasi tersebut dibandingkan variasi Naive Bayes lainnya? 

> Jawaban:

> Data yang kita miliki mayoritas bersifat numerik dan kontinu sehingga cocok untuk menggunakan `GaussianNB()`. Adapula beberapa yang memiliki nilai biner seperti `blue`, `dual_sim`, `fc` dll. tetapi mayoritas data bersifat kontinu sehingga di ambil `GaussianNB()`. Selain itu adapun melakukan fitting dengan model `MultinomialNB()` yang mengembalikan hasil lebih buruk dibandingkan gaussian.

16\. [6] Masing-masing variasi Naive Bayes dibuat khusus untuk menyesuaikan tipe data tertentu pada data yang digunakan. Jika data yang ada memiliki tipe fitur yang bervariasi (numerik, biner, kategorik, dll), apakah kita bisa menggunakan salah satu variasi algoritma saja? Bagaimana strategi Anda dalam menganani kasus tersebut?

> Jawaban:

> Secara teori bila ada variasi fitur kita bisa memisahkannya berdasarkan tipenya. Setelah itu menghitung probabilitas target pada masing-masing variasi fitur dan mengalikan hasilnya. Dalam implementasi, kita bisa menggunakan dua model yang berbeda (seperti Multinomial dan Gaussian) kemudian menggabungkan probabilitasnya dan mem-fitnya dalam model baru. Cara lain adalah dengan mengubah variabel kategorikal menjadi continuous/diskret sehingga hanya perlu dilakukan fitting ke model Gaussian atau Multinomial, tetapi ini tidak bisa berlaku bila kita ingin menggunakan model Gaussian.

### [12] K-nearest Neighbors

17\. [4] Implementasikan model KNN dengan memberikan parameter nilai `n_neighbors=3`; serta berikan hasil evaluasinya!

In [17]:
KNN = KNeighborsClassifier(n_neighbors=3)

np.mean(cross_val_score(KNN, X_selected, y, cv=kf))

0.8686080132176788

18\. [8] Implementasikan model KNN dengan parameter yang didapatkan dari hasil Hyperparameter Tuning, serta berikan hasil evaluasinya!

In [18]:
param_KNN = [
  { 
    'n_neighbors': [8,9,10,11,12]
  } 
]

KNN_est = KNeighborsClassifier()

est_for_knn = GridSearchCV(estimator= KNN_est, 
                   param_grid=param_KNN, cv= kf)

est_for_knn.fit(X_selected, y)

est_for_knn.best_params_

{'n_neighbors': 11}

In [19]:
KNN_param = KNeighborsClassifier(**est_for_knn.best_params_)

np.mean(cross_val_score(KNN_param, X_selected, y, cv=kf))

0.8819936665289827

### [7] Analisis

19\. [7] Dari keempat variasi model (Decision Tree, Random Forest, Naive Bayes, dan KNN) yang sudah Anda coba, manakah model yang memiliki performa terbaik? Berikan analisis Anda!

> Jawaban:

> Model terbaik didapatkan dengan Random Forest. Performa model tersebut lebih bagus dibandingkan dengan model lain karena beberapa faktor

> Bila dibandingkan dengan **DecisionTree** Random Forest lebih bagus karena menggabungkan beberapa DecisionTree, dengan variasi fitur/target berbeda-beda pada masing-masing tree, dan mengambil hasil rata-ratanya. Dengan begini kita mendapatkan hasil yang lebih bagus dibandingkan satu DecisionTree Saja

> Bila dibandingkan dengan **NaiveBayes** Random Forest lebih bagus karena faktor naive bayes yang disebutkan pada nomor 15. Random Forest tidak perlu akomodasi tambahan untuk masalah tipe data beda, tidak seperti naive bayes. Karena ini Random Forest lebih bagus

> Model Random Forest hampir setara dalam performa dengan **KNN**. Alasan saya memilih Random Forest adalah karena hasil cross_val_score-nya yang rata-rata lebih bagus dibandingkan KNN dengan hyperparameter tuning.